# Algorithmen



In [ ]:
%run ./Muehle_Logic.ipynb


In [ ]:
%run ./Muehle_Utilities.ipynb

In [ ]:
%run ./Muehle_Heuristik.ipynb

Die Funktion `memoize(f)` aus der "Wissensbasierte Systeme"-Vorlesung nimmt ein Argument f und gibt eine Version dieser Funktion zurück, die alle Ergebnisse der Funktion f zwischenspeichert.

In [ ]:
def memoize(f):
    global Cache
    
    def f_memoized(*args):
        key = (args[0], args[1])
        if key in Cache:
            return Cache[key]
        result = f(*args)
        Cache[key] = result
        return result
    
    return f_memoized

## Minimax-Algorithmus

In [ ]:
import random
random.seed(1)

`value_minimax(State, player)`

In [ ]:
@memoize
def value_minimax(state, player, depth):
    if finished(to_list(state)):
        return utility(to_list(state), player)
    if depth == 0:
        return heuristic(state, player)
    o = opponent(player)
    depth -= 1
    return max([ -value_minimax(to_tupel(ns), o, depth) for ns in next_states(to_list(state), player) ])

`best_move_minimax(State, player)`

In [ ]:
def best_move_minimax(state, player, depth):
    ns          = next_states(state, player)
    best_value  = value_minimax(to_tupel(state), player, depth)
    best_moves  = [s for s in ns if -value_minimax(to_tupel(s), opponent(player), depth) == best_value]
    best_state  = random.choice(best_moves)
    return best_value, best_state

Die Funktion `minimax(State, Player)` wurde erstellt, um die Funktion `best_move_minimax(State, player)` nach Außen eindeutiger von `alpha_beta_pruning(State, Player)` abzugrenzen.

In [ ]:
def minimax(state, player, depth = 4):
    return(best_move_minimax(state, player, depth))

## Alpha-Beta-Pruning

In [ ]:
Cache = {}

`value_ab(State, player, alpha=-1, beta=1)` 

In [ ]:
def value_ab(state, player, alpha=-1, beta=1, depth = 4):
    #print('Value-depth: '+str(depth))
    global Cache
    #to_tupel
    state = to_tupel(state)
    if state in Cache:
        value, a, b = Cache[state]
        if a <= alpha and beta <= b:
            return value
        else:
            alpha = min(alpha, a)
            beta  = max(beta , b)
            value   = alphaBeta(state, player, alpha, beta, depth=depth)
            Cache[state] = value, alpha, beta
            return value
    else:
        value = alphaBeta(state, player, alpha, beta, depth=depth)
        Cache[state] = value, alpha, beta
        return value

`alphaBeta(State, player, alpha, beta)`

In [ ]:
def alphaBeta(state, player, alpha, beta, depth):
    #print('AB-depth: '+str(depth))
    #tolist 
    state = to_list(state)
    if finished(state):
        return utility(state, player)
    if depth == 0:
        return heuristic(state, player)
    value = alpha
    for ns in next_states(state, player):
        #print('Tiefe auf: '+str(depth)+' verringert')
        value = max(value, -value_ab(ns, opponent(player), -beta, -alpha, depth = depth-1))
        if value >= beta:
            return value
        alpha = max(value, alpha)
    return value

In [ ]:
def best_move_ab(state, player, depth = 4):
    ns         = next_states(state, player)
    best_value = value_ab(state, player, depth = depth)
    best_moves = [s for s in ns if -value_ab(s, opponent(player), depth = depth - 1) == best_value]
    best_state = random.choice(best_moves)
    return best_value, best_state

Die Funktion `alpha_beta_pruning(State, Player)` wurde erstellt, um die Funktion `best_move(State, player)` nach Außen eindeutiger von `minimax(State, Player)` abzugrenzen.

In [ ]:
def alpha_beta_pruning(state, player, depth = 4):
    return(best_move_ab(state, player, depth = depth))

## Funktionstests:

In [ ]:
import time
start = time.time()
state = [[4, 5], [[0, 2, 0, 0, 2, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0], [2, 1, 1, 1, 0, 0, 0, 0]]]
print(alpha_beta_pruning(state, 2, 5))
end = time.time()
print(str(end-start)+'sec')

In [ ]:
import time
start = time.time()
state = [[4, 5], [[0, 2, 0, 0, 2, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0], [2, 1, 1, 1, 0, 0, 0, 0]]]
print(minimax(state, 2, 5))
end = time.time()
print(str(end-start)+'sec')

In [ ]:
import time
start = time.time()
state = [[4, 5], [[0, 2, 0, 0, 2, 0, 0, 0], [2, 1, 1, 1, 0, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0]]]
print(alpha_beta_pruning(state, 2, 5))
end = time.time()
print(str(end-start)+'sec')